<a href="https://colab.research.google.com/github/Dev-Dhanush-hub/Computer_vision_finalproject/blob/main/stereo3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install dependencies
!pip install open3d==0.17.0
!pip install imageio scikit-image gdown

# Step 2: Import modules
import os
import cv2
import gdown
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from PIL import Image


In [ ]:
# Download stereo pair from Google Drive
os.makedirs("data", exist_ok=True)
gdown.download("https://drive.google.com/uc?id=1wMtfweRhZvo5KHvJIfY-eBMEKj69AkgO", "data/left.png", quiet=False)
gdown.download("https://drive.google.com/uc?id=1uvOAVZ_mZstfqJgCybKoqz6zB6KT9XW2", "data/right.png", quiet=False)

# Load images
left = cv2.imread("data/left.png")
right = cv2.imread("data/right.png")

# Convert to grayscale for stereo matching
left_gray = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
right_gray = cv2.cvtColor(right, cv2.COLOR_BGR2GRAY)

# Show input
plt.subplot(1,2,1), plt.imshow(left[..., ::-1]), plt.title("Left Image")
plt.subplot(1,2,2), plt.imshow(right[..., ::-1]), plt.title("Right Image")
plt.show()


In [ ]:
# Create disparity map using OpenCV SGBM
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=96,
    blockSize=9,
    P1=8 * 3 * 9 ** 2,
    P2=32 * 3 * 9 ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=15,
    speckleWindowSize=50,
    speckleRange=2,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

disparity_map = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0

plt.imshow(disparity_map, cmap='plasma')
plt.colorbar()
plt.title("Disparity Map")
plt.show()


In [ ]:
# Create disparity map using OpenCV SGBM
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=96,
    blockSize=9,
    P1=8 * 3 * 9 ** 2,
    P2=32 * 3 * 9 ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=15,
    speckleWindowSize=50,
    speckleRange=2,
    preFilterCap=63,
    mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
)

disparity_map = stereo.compute(left_gray, right_gray).astype(np.float32) / 16.0

plt.imshow(disparity_map, cmap='plasma')
plt.colorbar()
plt.title("Disparity Map")
plt.show()


In [ ]:
# Reproject to 3D using fake camera matrix Q (adjust for real use)
h, w = disparity_map.shape
f = 0.8 * w  # approximate focal length
Q = np.float32([[1, 0, 0, -w/2],
                [0, -1, 0, h/2],
                [0, 0, 0, -f],
                [0, 0, 1/0.1, 0]])  # baseline = 0.1m

points_3d = cv2.reprojectImageTo3D(disparity_map, Q)
colors = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
mask = disparity_map > disparity_map.min()

output_points = points_3d[mask]
output_colors = colors[mask]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(output_points)
pcd.colors = o3d.utility.Vector3dVector(output_colors / 255.0)

# Save point cloud
o3d.io.write_point_cloud("scene.ply", pcd)


In [ ]:
# Visualize interactively in Colab
o3d.visualization.draw_geometries([pcd])
